In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from plato.planets.metrics import PlanetPopulationMetrics
from plato.planets.populations import PopulationModel
from plato.stars import filter_p1_targets, filter_valid_targets
from plato.visualisation import (
    FigureProcessor,
    contour_plot,
    get_earth_marker,
    get_palette,
    set_plot_defaults,
)

In [3]:
# set_plot_defaults()

figure_directory = f"../figures/06_planet_populations/"
save = True

In [4]:
LOPS2 = pd.read_csv(f"../data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(f"../data/processed/LOPN1_targets.csv")

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields)
fields = fields[
    [
        "Radius",
        "Mass",
        "[Fe/H]",
        "u1",
        "u2",
        "gaiaV",
        "n_cameras",
        "Population",
    ]
]

fields["cos_i"] = 0
fields["sigma_star"] = 10e-6
fields = fields.rename(
    columns={
        "Radius": "R_star",
        "Mass": "M_star",
        "gaiaV": "Magnitude_V",
    }
)

p1_sample = filter_p1_targets(fields)

In [5]:
metrics = PlanetPopulationMetrics()

## Recreate NGPPS II Table 7

In [6]:
population_model = PopulationModel(
    fields,
    num_embryos=100,
    additional_columns=["Population"],
)

In [7]:
mock = population_model.create_mock_population(additional_columns=["Population"])

In [8]:
table = metrics.calculate_metrics(
    mock,
    by_population=True,
    number_of_systems={
        pop: len(fields[fields["Population"] == pop])
        for pop in fields["Population"].unique()
    },
)
table.reindex(
    index=pd.MultiIndex.from_product(
        [
            [
                "Thin Disk",
                "Thick Disk Candidate",
                "Thick Disk",
                "Halo Candidate",
                "Halo",
            ],
            ["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"],
        ],
    )
)

Number of Planets  System Fraction  \
Thin Disk            Earth                   790159         0.888286   
                     Super-Earth             657768         0.752128   
                     Neptunian                60397         0.241449   
                     Sub-Giant                17242         0.075930   
                     Giant                    46214         0.171425   
Thick Disk Candidate Earth                   372097         0.882867   
                     Super-Earth             305858         0.739976   
                     Neptunian                28115         0.233829   
                     Sub-Giant                 8463         0.077161   
                     Giant                    22609         0.176333   
Thick Disk           Earth                    45093         0.897947   
                     Super-Earth              23839         0.635510   
                     Neptunian                 1848         0.151339   
                     Sub-Giant                  482         0.042561   
                     Giant                     1319         0.096370   
Halo Candidate       Earth                     2401         0.891892   
                     Super-Earth                475         0.452088   
                     Neptunian                   14         0.022113   
                     Sub-Giant                    3         0.004914   
                     Giant                       12         0.017199   
Halo                 Earth                     2952         0.891348   
                     Super-Earth                473         0.418511   
                     Neptunian                   11         0.014085   
                     Sub-Giant                    9         0.010060   
                     Giant                       24         0.032193   

                                  Occurrence Rate  Multiplicity  
Thin Disk            Earth               4.588665      5.165755  
                     Super-Earth         3.819835      5.078701  
                     Neptunian           0.350742      1.452654  
                     Sub-Giant           0.100129      1.318700  
                     Giant               0.268377      1.565568  
Thick Disk Candidate Earth               4.525021      5.125374  
                     Super-Earth         3.719498      5.026508  
                     Neptunian           0.341903      1.462191  
                     Sub-Giant           0.102917      1.333806  
                     Giant               0.274945      1.559241  
Thick Disk           Earth               5.229387      5.823712  
                     Super-Earth         2.764583      4.350182  
                     Neptunian           0.214311      1.416092  
                     Sub-Giant           0.055897      1.313351  
                     Giant               0.152963      1.587244  
Halo Candidate       Earth               5.899263      6.614325  
                     Super-Earth         1.167076      2.581522  
                     Neptunian           0.034398      1.555556  
                     Sub-Giant           0.007371      1.500000  
                     Giant               0.029484      1.714286  
Halo                 Earth               5.939638      6.663657  
                     Super-Earth         0.951710      2.274038  
                     Neptunian           0.022133      1.571429  
                     Sub-Giant           0.018109      1.800000  
                     Giant               0.048290      1.500000

## GET UNCERTAINTIES FOR HALO PLANETS, ALSO METAllICITY CUT

In [9]:
halo_population_model = PopulationModel(
    fields[fields["Population"] == "Halo"],
    num_embryos=100,
    additional_columns=["Population"],
)

In [11]:
halo_mocks = [halo_population_model.create_mock_population() for _ in range(300)]
halo_metrics = [
    metrics.calculate_metrics(
        mock,
        number_of_systems=sum(fields["Population"] == "Halo"),
    )
    for mock in halo_mocks
]
halo_table = metrics.calculate_metrics_stats(
    halo_metrics, grouping_column="Planet Category"
)

halo_table.reindex(["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"])

Number of Planets               System Fraction            \
                           Median   16th   84th          Median      16th   
Planet Category                                                             
Earth                      2928.0  72.48  75.00        0.895372  0.016097   
Super-Earth                 554.0  45.00  37.16        0.426559  0.022455   
Neptunian                    19.0   5.00   6.00        0.024145  0.008048   
Sub-Giant                    14.0   5.00   5.00        0.016097  0.006036   
Giant                        24.0   5.00   4.00        0.032193  0.006036   

                          Occurrence Rate                     Multiplicity  \
                     84th          Median      16th      84th       Median   
Planet Category                                                              
Earth            0.012072        5.891348  0.145835  0.150905     6.584283   
Super-Earth      0.018109        1.114688  0.090543  0.074769     2.613747   
Neptunian        0.006036        0.038229  0.010060  0.012072     1.666667   
Sub-Giant        0.006036        0.028169  0.010060  0.010060     1.750000   
Giant            0.006036        0.048290  0.010060  0.008048     1.454545   

                                     
                     16th      84th  
Planet Category                      
Earth            0.120285  0.137107  
Super-Earth      0.159845  0.169107  
Neptunian        0.205128  0.224000  
Sub-Giant        0.194444  0.150000  
Giant            0.138756  0.133690

In [12]:
halo_cut_mocks = [
    halo_population_model.create_mock_population(metallicity_limit=-0.6)
    for _ in range(300)
]
halo_cut_metrics = [
    metrics.calculate_metrics(
        mock,
        number_of_systems=sum(fields["Population"] == "Halo"),
    )
    for mock in halo_cut_mocks
]
halo_cut_table = metrics.calculate_metrics_stats(
    halo_cut_metrics, grouping_column="Planet Category"
)
halo_cut_table.reindex(["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"])

Number of Planets             System Fraction            \
                           Median  16th  84th          Median      16th   
Planet Category                                                           
Earth                       384.0  25.0  25.0        0.156942  0.008048   
Super-Earth                 258.5  26.5  28.5        0.114688  0.008048   
Neptunian                    20.0   6.0   5.0        0.022133  0.004024   
Sub-Giant                    13.0   4.0   5.0        0.015091  0.005030   
Giant                        23.0   5.0   5.0        0.032193  0.006036   

                          Occurrence Rate                     Multiplicity  \
                     84th          Median      16th      84th       Median   
Planet Category                                                              
Earth            0.006036        0.772636  0.050302  0.050302     4.937886   
Super-Earth      0.008048        0.520121  0.053320  0.057344     4.563329   
Neptunian        0.008048        0.040241  0.012072  0.010060     1.666667   
Sub-Giant        0.005030        0.026157  0.008048  0.010060     1.769231   
Giant            0.006036        0.046278  0.010060  0.010060     1.444444   

                                     
                     16th      84th  
Planet Category                      
Earth            0.246528  0.275745  
Super-Earth      0.383466  0.381433  
Neptunian        0.205128  0.209510  
Sub-Giant        0.169231  0.119658  
Giant            0.131944  0.118056

## RECREATING NGPPS II TABLE 6 (INFLUENCE OF EMBRYOS)

In [11]:
embryo_table = []

for num_embryos in [10, 20, 50, 100]:
    model = PopulationModel(
        fields,
        num_embryos=num_embryos,
        additional_columns=["Population"],
    )

    mock_population = model.create_mock_population(additional_columns=["Population"])

    metric_table = metrics.calculate_number_of_planets(
        mock_population, by_population=True
    )
    embryo_table.append(metric_table)

embryo_table = pd.concat(embryo_table, axis=1).rename(
    {0: "10", 1: "20", 2: "50", 3: "100"}, axis=1
)

embryo_table.reindex(
    index=pd.MultiIndex.from_product(
        [
            [
                "Thin Disk",
                "Thick Disk Candidate",
                "Thick Disk",
                "Halo Candidate",
                "Halo",
            ],
            ["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"],
        ],
    )
)

In [10]:
pd.concat(table, axis=1).rename({0: "10", 1: "20", 2: "50", 3: "100"}, axis=1).reindex(
    index=pd.MultiIndex.from_product(
        [
            [
                "Thin Disk",
                "Thick Disk Candidate",
                "Thick Disk",
                "Halo Candidate",
                "Halo",
            ],
            ["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"],
        ],
    )
)

10      20      50     100
Thin Disk            Earth        193119  353817  659657  789860
                     Super-Earth  247135  341798  535998  657690
                     Neptunian     60660   64054   55891   60153
                     Sub-Giant     25111   20417   15743   17035
                     Giant         46124   47485   43656   45777
Thick Disk Candidate Earth         90088  168127  307020  372958
                     Super-Earth  118126  162389  247041  304663
                     Neptunian     28087   29850   26125   28109
                     Sub-Giant     12228    9569    7403    8491
                     Giant         21661   22050   21612   22723
Thick Disk           Earth          9429   18115   35193   45007
                     Super-Earth   15799   19592   19887   23866
                     Neptunian      2836    3491    1598    1739
                     Sub-Giant      1677    1198     439     487
                     Giant          1856    1896    1115    1268
Halo Candidate       Earth           345     873    1637    2397
                     Super-Earth    1074    1011     388     537
                     Neptunian        95     183      18      13
                     Sub-Giant       133      52       5       2
                     Giant            47      45       7       6
Halo                 Earth           461     892    1990    2813
                     Super-Earth    1365    1431     494     616
                     Neptunian        99     209      20      18
                     Sub-Giant       173      57       4      16
                     Giant            40      26      26      23

## OCCURENCE RATE PLOT